In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

VQE with parallel QPUs with Rigetti
===================================

::: {.meta}
:property=\"og:description\": Using parallel QPUs to speed up the
calculation of the potential energy surface of molecular Hamiltonian.
:property=\"og:image\":
<https://pennylane.ai/qml/_images/vqe_diagram.png>
:::

::: {.related}
tutorial\_vqe A brief overview of VQE
:::

*Author: Tom Bromley --- Posted: 14 February 2020. Last updated: 29
August 2023.*

This tutorial showcases how using asynchronously-evaluated parallel QPUs
can speed up the calculation of the potential energy surface of
molecular hydrogen ($H_2$).

Using a VQE setup, we task two devices from the
[PennyLane-Rigetti](https://docs.pennylane.ai/projects/rigetti/en/latest/)
plugin with evaluating separate terms in the qubit Hamiltonian of $H_2$.
As these devices are allowed to operate asynchronously, i.e., at the
same time and without having to wait for each other, the calculation can
be performed in roughly half the time.

We begin by importing the prerequisite libraries:


This tutorial requires the `pennylane-rigetti` and `dask` packages,
which are installed separately using:

``` {.bash}
pip install pennylane-rigetti
pip install "dask[delayed]"
```

Finding the qubit Hamiltonians of $H_{2}$
=========================================

The objective of this tutorial is to evaluate the potential energy
surface of molecular hydrogen. This is achieved by finding the ground
state energy of $H_{2}$ as we increase the bond length between the
hydrogen atoms.

Each inter-atomic distance results in a different qubit Hamiltonian.
Further details on the mapping from the electronic Hamiltonian of a
molecule to a qubit Hamiltonian can be found in the
`tutorial_quantum_chemistry`{.interpreted-text role="doc"} and
`tutorial_vqe`{.interpreted-text role="doc"} tutorials.

We begin by downloading a selection of datasets of $H_2$ molecule for
various bond lengths using the [PennyLane Datasets
library](https://pennylane.ai/datasets/qchem/h2-molecule):


We can now extract the qubit Hamiltonians from these datasets for each
bond length:


Each Hamiltonian can be written as a linear combination of fifteen
tensor products of Pauli matrices. Let\'s take a look more closely at
one of the Hamiltonians:


::: {.rst-class}
sphx-glr-script-out

``` {.none}
Number of terms: 15

Measurement Identity on wires <Wires = [0]>
Measurement PauliZ on wires <Wires = [0]>
Measurement PauliZ on wires <Wires = [1]>
Measurement ['PauliZ', 'PauliZ'] on wires <Wires = [0, 1]>
Measurement ['PauliY', 'PauliX', 'PauliX', 'PauliY'] on wires <Wires = [0, 1, 2, 3]>
Measurement ['PauliY', 'PauliY', 'PauliX', 'PauliX'] on wires <Wires = [0, 1, 2, 3]>
Measurement ['PauliX', 'PauliX', 'PauliY', 'PauliY'] on wires <Wires = [0, 1, 2, 3]>
Measurement ['PauliX', 'PauliY', 'PauliY', 'PauliX'] on wires <Wires = [0, 1, 2, 3]>
Measurement PauliZ on wires <Wires = [2]>
Measurement ['PauliZ', 'PauliZ'] on wires <Wires = [0, 2]>
Measurement PauliZ on wires <Wires = [3]>
Measurement ['PauliZ', 'PauliZ'] on wires <Wires = [0, 3]>
Measurement ['PauliZ', 'PauliZ'] on wires <Wires = [1, 2]>
Measurement ['PauliZ', 'PauliZ'] on wires <Wires = [1, 3]>
Measurement ['PauliZ', 'PauliZ'] on wires <Wires = [2, 3]>
```
:::


Defining the energy function
============================

The fifteen Pauli terms comprising each Hamiltonian can conventionally
be evaluated in a sequential manner: we evaluate one expectation value
at a time before moving on to the next. However, this task is highly
suited to parallelization. With access to multiple QPUs, we can split up
evaluating the terms between the QPUs and gain an increase in processing
speed.

::: {.note}
::: {.title}
Note
:::

Some of the Pauli terms commute, and so they can be evaluated in
practice with fewer than fifteen quantum circuit runs. Nevertheless,
these quantum circuit runs can still be parallelized to multiple QPUs.
:::

Let\'s suppose we have access to two quantum devices. In this tutorial
we consider two simulators from Rigetti: `4q-qvm` and `9q-square-qvm`,
but we could also run on hardware devices from Rigetti or other
providers.

We can evaluate the expectation value of each Hamiltonian with eight
terms run on one device and seven terms run on the other, as summarized
by the diagram below:

![](/demonstrations/vqe_parallel/vqe_diagram.png){.align-center
width="65.0%"}

To do this, start by instantiating a device for each term:


::: {.note}
::: {.title}
Note
:::

For the purposes of this demonstration, we are simulating the QPUs using
the `rigetti.qvm` simulator. To run this demonstration on hardware,
simply swap `rigetti.qvm` for `rigetti.qpu` and specify the hardware
device to run on.

Please refer to the [Rigetti website](https://rigetti.com/) for an
up-to-date list on available QPUs.
:::

::: {.warning}
::: {.title}
Warning
:::

Rigetti\'s QVM and Quil Compiler services must be running for this
tutorial to execute. They can be installed by consulting the [Rigetti
documentation](http://docs.rigetti.com/qcs/) or, for users with Docker,
by running:

``` {.bash}
docker run -d -p 5555:5555 rigetti/quilc -R -p 5555
docker run -d -p 5000:5000 rigetti/qvm -S -p 5000
```
:::

We must also define a circuit to prepare the ground state, which is a
superposition of the Hartree-Fock ($|1100\rangle$) and doubly-excited
($|0011\rangle$) configurations. The simple circuit below is able to
prepare states of the form $\alpha |1100\rangle +
\beta |0011\rangle$ and hence encode the ground state wave function of
the hydrogen molecule. The circuit has a single free parameter, which
controls a Y-rotation on the third qubit.


The ground state for each inter-atomic distance is characterized by a
different Y-rotation angle. The values of these Y-rotations can be found
by minimizing the ground state energy as outlined in
`tutorial_vqe`{.interpreted-text role="doc"}. In this tutorial, we load
pre-optimized rotations and focus on comparing the speed of evaluating
the potential energy surface with sequential and parallel evaluation.
These parameters can be downloaded by clicking `here
<../demonstrations/vqe_parallel/RY_params.npy>`{.interpreted-text
role="download"}.


Calculating the potential energy surface
========================================

The most vanilla execution of these 10 energy surfaces is using the
standard PennyLane functionalities by executing the QNodes. Internally,
this creates a measurement for each term in the Hamiltonian that are
then sequentially computed.


We can parallelize the individual evaluations using `dask` in the
following way: We take the 15 terms of the Hamiltonian and distribute
them to the 15 devices in `devs`. This evaluation is delayed using
`dask.delayed` and later computed in parallel using `dask.compute`,
which asynchronously executes the delayed objects in `results`.


We can now compute all 10 samples from the energy surface sequentially,
where each execution is making use of parallel device execution.
Curiously, in this example the overhead from doing so outweighs the
speed-up and the execution is slower than standard execution using
`qml.expval`. For different circuits and different Hamiltonians,
however, parallelization may provide significant speed-ups.


We can improve this procedure further by optimizing the measurement.
Currently, we are measuring each term of the Hamiltonian in a separate
measurement. This is not necessary as there are sub-groups of commuting
terms in the Hamiltonian that can be measured simultaneously. We can
utilize the grouping function
`~.pennylane.pauli.group_observables`{.interpreted-text role="func"} to
generate few measurements that are executed in parallel:


::: {.rst-class}
sphx-glr-script-out

``` {.none}
Evaluating the potential energy surface sequentially
1 / 10: Sequential execution; Running for inter-atomic distance 0.3 Å
2 / 10: Sequential execution; Running for inter-atomic distance 0.5 Å
3 / 10: Sequential execution; Running for inter-atomic distance 0.7 Å
4 / 10: Sequential execution; Running for inter-atomic distance 0.9 Å
5 / 10: Sequential execution; Running for inter-atomic distance 1.1 Å
6 / 10: Sequential execution; Running for inter-atomic distance 1.3 Å
7 / 10: Sequential execution; Running for inter-atomic distance 1.5 Å
8 / 10: Sequential execution; Running for inter-atomic distance 1.7 Å
9 / 10: Sequential execution; Running for inter-atomic distance 1.9 Å
10 / 10: Sequential execution; Running for inter-atomic distance 2.1 Å
Evaluation time: 39.33 s

Evaluating the potential energy surface in parallel
1 / 10: Parallel execution; Running for inter-atomic distance 0.3 Å
2 / 10: Parallel execution; Running for inter-atomic distance 0.5 Å
3 / 10: Parallel execution; Running for inter-atomic distance 0.7 Å
4 / 10: Parallel execution; Running for inter-atomic distance 0.9 Å
5 / 10: Parallel execution; Running for inter-atomic distance 1.1 Å
6 / 10: Parallel execution; Running for inter-atomic distance 1.3 Å
7 / 10: Parallel execution; Running for inter-atomic distance 1.5 Å
8 / 10: Parallel execution; Running for inter-atomic distance 1.7 Å
9 / 10: Parallel execution; Running for inter-atomic distance 1.9 Å
10 / 10: Parallel execution; Running for inter-atomic distance 2.1 Å
Evaluation time: 73.42 s

Evaluating the potential energy surface in parallel with measurement optimization
1 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 0.3 Å
2 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 0.5 Å
3 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 0.7 Å
4 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 0.9 Å
5 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 1.1 Å
6 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 1.3 Å
7 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 1.5 Å
8 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 1.7 Å
9 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 1.9 Å
10 / 10: Parallel execution and measurement optimization; Running for inter-atomic distance 2.1 Å
Evaluation time: 26.51 s
```
:::


We have seen how Hamiltonian measurements can be parallelized and
optimized at the same time.


::: {.rst-class}
sphx-glr-script-out

``` {.none}
Speed up: 1.48
```
:::


To conclude the tutorial, let\'s plot the calculated potential energy
surfaces:


![](/demonstrations/vqe_parallel/vqe_parallel_001.png){.align-center
width="80.0%"}


These surfaces overlap, with any variation due to the limited number of
shots used to evaluate the expectation values in the `rigetti.qvm`
device (we are using the default value of `shots=1024`).


About the author
================
